In [1]:
import ccf2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import gc
import warnings
import lightgbm as lgb
import xgboost
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split,cross_validate
from sklearn.metrics import mean_squared_error as mse
from sklearn.grid_search import GridSearchCV
warnings.filterwarnings('ignore')

path = '../input/'

D:\programfiles\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
D:\programfiles\anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# load data

In [2]:
data = ccf2.loaddata(path)

In [3]:
data.shape

(36960, 12)

# feature

## 构建特征

In [4]:
## 平移特征
data,shift_feat = ccf2.genShitFeat(data,[12])

In [5]:
## 单列分组特征
fea_list = ['adcode','bodyType','model','regMonth','mt']
data,stat_feat = ccf2.genStatFeat(data,fea_list)

统计列的sum,mean,max,min,分位数0.2,分位数0.5,分位数0.8
构造特征: adcode
构造特征: bodyType
构造特征: model
构造特征: regMonth
构造特征: mt


In [6]:
## 多列分组特征
data['adcode_model'] = data['adcode']+data['model']
data['adcode_bodyType'] = data['adcode']+data['bodyType']
data['adcode_month'] = data['adcode']+data['regMonth']
data['adcode_mt'] = data['adcode']+data['mt']

data['bodyType_month'] = data['bodyType']+data['regMonth']
data['bodyType_mt'] = data['bodyType']+data['mt']
data['model_month'] = data['model']+data['regMonth']
data['model_mt'] = data['model']+data['mt']

# data['adcode_model_month'] = data['adcode_model']*100+data['regMonth']
# data['adcode_model_mt'] = data['adcode_model']*100+data['mt']
# data['adcode_bodyType_month'] = data['adcode_bodyType']*100+data['regMonth']
# data['adcode_bodyType_mt'] = data['adcode_bodyType']*100+data['mt']

fea_list2 = ['adcode_model','adcode_bodyType','adcode_month','adcode_mt',
            'bodyType_month','bodyType_mt','model_month','model_mt'] 

data,stat_feat2 = ccf2.genStatFeat(data,fea_list2)

统计列的sum,mean,max,min,分位数0.2,分位数0.5,分位数0.8
构造特征: adcode_model
构造特征: adcode_bodyType
构造特征: adcode_month
构造特征: adcode_mt
构造特征: bodyType_month
构造特征: bodyType_mt
构造特征: model_month
构造特征: model_mt


In [7]:
num_feat = ['regYear']
cate_feat1 = ['adcode', 'bodyType', 'model', 'regMonth']
cate_feat = cate_feat1+fea_list+fea_list2
features = cate_feat1+num_feat+stat_feat+stat_feat2+shift_feat

In [8]:
data.shape

(36960, 115)

## label

In [9]:
import math
# data['model_weight'] = data.groupby('model')['label'].transform('mean') 
# data['n_label'] = data['label'] / data['model_weight'] 
data['log_label'] = data['label'].apply(lambda x: math.log(x))

# 构建模型

## 拆分数据

In [10]:
train_idx = (data['mt'] <= 24)
test_idx = (data['mt'] > 24)

trainSet = data.loc[train_idx]
testSet = data.loc[test_idx]
# train_x = data[train_idx][features]
# train_y = data[train_idx]['n_label']
# train_y2 = data[train_idx]['label']
# valid_idx = (data['mt'].between(21, 24))
# valid_x = data[valid_idx][features]
# valid_y = data[valid_idx]['n_label']

In [29]:
trainSet.head()

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,...,adcode_bodyType_month_median5,adcode_bodyType_month_median8,adcode_bodyType_mt_sum,adcode_bodyType_mt_mean,adcode_bodyType_mt_max,adcode_bodyType_mt_mim,adcode_bodyType_mt_median2,adcode_bodyType_mt_median5,adcode_bodyType_mt_median8,log_label
0,310000,0,0,0,上海,1,2016,1479.0,11.0,106.0,...,312.0,452.0,11822.0,472.88,1572.0,165.0,291.4,374.0,486.2,5.676754
1,530000,0,0,0,云南,1,2016,1594.0,11.0,106.0,...,383.0,519.4,11308.0,452.32,1704.0,77.0,238.6,351.0,624.8,6.144186
2,150000,0,0,0,内蒙古,1,2016,1479.0,11.0,106.0,...,259.5,471.8,8204.0,328.16,939.0,50.0,165.8,260.0,471.8,5.549076
3,110000,0,0,0,北京,1,2016,2370.0,11.0,106.0,...,435.5,760.2,15517.0,620.68,1119.0,130.0,407.4,541.0,900.8,6.011267
4,510000,0,0,0,四川,1,2016,3562.0,11.0,106.0,...,670.0,1230.8,24635.0,985.40,3386.0,222.0,430.8,739.0,1459.8,6.413459


## cv 交叉验证

In [11]:
lgb_model = lgb.LGBMRegressor(
        num_leaves=32, reg_alpha=1, reg_lambda=0.1, objective='mse',
        max_depth=-1, learning_rate=0.05, min_child_samples=5, random_state=np.random.randint(1000),
        n_estimators=1000, subsample=0.8, colsample_bytree=0.8,
    )

In [34]:
# from sklearn.utils.multiclass import type_of_target
# type_of_target(train_y)

In [12]:
gc.collect()

210

In [13]:
n_split = 5
kfd = KFold(n_splits=n_split,random_state=1992)
# res_prob = pd.DataFrame()
res = pd.DataFrame()

index = 1
for train_idx,test_idx in kfd.split(trainSet):
    print('-------------------------------','Kflod:',index,'----------------------------')
    
    train_x,train_y = trainSet.loc[train_idx][features],trainSet.loc[train_idx]['log_label']
    valid_x,valid_y = trainSet.loc[test_idx][features],trainSet.loc[test_idx]['log_label']
    lgb_model.fit(train_x,train_y, 
                  eval_set=[(train_x,train_y),
                            (valid_x,valid_y)],
                  categorical_feature=cate_feat, 
                  early_stopping_rounds=50,
                  verbose=100)
#     x_pred = lgb_model.predict(train_x[test_idx], num_iteration=lgb_model.best_iteration_)
    gc.collect()
    test_pred = lgb_model.predict(testSet[features], num_iteration=lgb_model.best_iteration_)
    res['prob_%s' % str(index)] = test_pred
    index = index+1

------------------------------- Kflod: 1 ----------------------------
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.033445	valid_1's l2: 0.13663
[200]	training's l2: 0.0180686	valid_1's l2: 0.13629
[300]	training's l2: 0.012797	valid_1's l2: 0.136014
Early stopping, best iteration is:
[337]	training's l2: 0.0115603	valid_1's l2: 0.135801
------------------------------- Kflod: 2 ----------------------------
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.0359956	valid_1's l2: 0.0780015
Early stopping, best iteration is:
[69]	training's l2: 0.0509815	valid_1's l2: 0.0767884
------------------------------- Kflod: 3 ----------------------------
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.0355738	valid_1's l2: 0.142616
[200]	training's l2: 0.0192585	valid_1's l2: 0.14048
[300]	training's l2: 0.0134055	valid_1's l2: 0.139386
[400]	training's l2: 0.00991565	valid_1's l2: 0.1391

## 生成提交文件

In [14]:
res.head()

,prob_1,prob_2,prob_3,prob_4,prob_5
0,5.685555,5.603918,5.571715,5.681709,5.520856
1,5.874216,5.684645,5.733518,5.926372,5.649046
2,5.298278,5.188856,5.153938,5.322710,5.165938
3,6.039564,5.776816,5.923146,6.032821,5.824777
4,6.144074,6.028993,5.966230,6.211716,6.083248


In [15]:
sub = ccf2.genLogSub(data,res,n_split)

In [16]:
sub.to_csv(path+'sub/sub_cv3.csv',index=False,header=True)

## 特征重要性分析

In [22]:
df_imp = pd.DataFrame()
imp = lgb_model.feature_importances_
df_imp['features'] = features
df_imp['import'] = imp
df_imp.sort_values(by=['import'],ascending=False,inplace=True)

In [27]:
df_imp.head()

,features,import
22,model,5714
6,adcode,3678
5,shift_model_adcode_mt_label_12,3170
2,model,896
30,regMonth,889


In [17]:
df_imp = ccf2.featImport(lgb_model,features)

In [30]:
df_imp.head()

,features,import
119,adcode_model_mt_sum,1947
120,adcode_model_mt_mean,343
114,adcode_model_month_mim,270
138,adcode_bodyType_mt_mim,200
46,adcode_model,135


In [18]:
df_imp.head()

,features,import
21,model,483
69,adcode_mt,367
101,model_mt,325
93,model_month,231
46,adcode_model_sum,227
